In [1]:
import pandas as pd

In [2]:
cod_campus = None
cod_instituicao = 588
edicoes = [(2016, 1), (2016, 2), (2017, 1)]#, (2017, 2), (2018, 1), (2018, 2), (2019, 1), (2019, 2), (2020, 1), (2020, 2), (2021, 1),(2021, 2), (2022, 1), (2022, 2)]

In [3]:
bases_sisu = {
    "2016-1": {"ano": 2016, "semestre": 1, "encoding": "utf-8", "delimiter": "|"},
    "2016-2": {"ano": 2016, "semestre": 2, "encoding": "utf-8", "delimiter": "|"},
    "2017-1": {"ano": 2017, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2017-2": {"ano": 2017, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2018-1": {"ano": 2018, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2018-2": {"ano": 2018, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2019-1": {"ano": 2019, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2019-2": {"ano": 2019, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2020-1": {"ano": 2020, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2020-2": {"ano": 2020, "semestre": 2, "encoding": "utf-8", "delimiter": ";"},
    "2021-1": {"ano": 2021, "semestre": 1, "encoding": "utf-8", "delimiter": ";"},
    "2021-2": {"ano": 2021, "semestre": 2, "encoding": "cp1252", "delimiter": "|"},
    "2022-1": {"ano": 2022, "semestre": 1, "encoding": "cp1252", "delimiter": "|"},
    "2022-2": {"ano": 2022, "semestre": 2, "encoding": "cp1252", "delimiter": "|"},
}

In [4]:
def selecionar_colunas(chave, dados_sisu, dados_enem):

    enem = dados_enem[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'TP_SEXO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q022', 'Q024', 'Q025']].copy()

    enem = enem.rename(columns={'NU_NOTA_CN': 'CN', 'NU_NOTA_CH': 'CH', 'NU_NOTA_LC': 'LC', 'NU_NOTA_MT': 'MT', 'NU_NOTA_REDACAO': 'REDACAO', 'TP_SEXO': 'SEXO'})

    if chave in ["2016-1", "2016-2"]:
        sisu = dados_sisu[['nu_nota_cn', 'nu_nota_ch', 'nu_nota_l', 'nu_nota_m', 'nu_nota_r', 'nota_concorrencia', 'nu_notacorte', 'tp_sexo', 'nome_curso', 'dt_nascimento', 'uf_origem', 'municipio_candidato', 'st_aprovado', 'nu_ano', 'nu_edicao', 'sigla_ies', 'campus']].copy()
        sisu = sisu.rename(columns={'nu_nota_l': 'LC', 'nu_nota_ch': 'CH', 'nu_nota_cn': 'CN', 'nu_nota_m': 'MT', 'nu_nota_r': 'REDACAO'})
        
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = dados_sisu[['NOTA_CN', 'NOTA_CH', 'NOTA_L', 'NOTA_M', 'NOTA_R', 'NOTA_CANDIDATO', 'NOTA_CORTE', 'SEXO', 'NOME_CURSO', 'DATA_NASCIMENTO', 'UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'APROVADO','ANO', 'EDICAO', 'SIGLA_IES', 'NOME_CAMPUS']].copy()
        sisu = sisu.rename(columns={'NOTA_L': 'LC', 'NOTA_CH': 'CH', 'NOTA_CN': 'CN', 'NOTA_M': 'MT', 'NOTA_R': 'REDACAO'})

    else:
        sisu = dados_sisu[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_L', 'NU_NOTA_M', 'NU_NOTA_R', 'NU_NOTA_CANDIDATO', 'NU_NOTACORTE', 'TP_SEXO', 'NO_CURSO', 'DT_NASCIMENTO', 'SG_UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'ST_APROVADO', 'NU_ANO', 'NU_EDICAO', 'SIGLA_IES', 'NO_CAMPUS']].copy()
        sisu = sisu.rename(columns={'NU_NOTA_L': 'LC', 'NU_NOTA_CH': 'CH', 'NU_NOTA_CN': 'CN', 'NU_NOTA_M': 'MT', 'NU_NOTA_R': 'REDACAO'})

    return sisu, enem

In [5]:
def filtrar_campus(chave, sisu, cod_campus):
    if chave in ["2016-1", "2016-2"]:
        sisu = sisu[sisu["cod_campus"] == cod_campus]
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = sisu[sisu['CODIGO_CAMPUS'] == cod_campus]
    else:
        sisu = sisu[sisu['CO_CAMPUS'] == cod_campus]
    
    return sisu

In [6]:
def filtrar_instituicao(chave, sisu, cod_instituicao):
    if chave in ["2016-1", "2016-2"]:
        sisu = sisu[sisu["cod_ies"] == cod_instituicao]
    elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
        sisu = sisu[sisu['CODIGO_IES'] == cod_instituicao]
    else:
        sisu = sisu[sisu['CO_IES'] == cod_instituicao]
    
    return sisu

In [7]:
def processar_base(chave, sisu, enem):
    
    colunas = ['CH', 'CN', 'MT', 'LC', 'REDACAO']
    
    for col in colunas:
        if col in sisu.columns:
            sisu.loc[:, col] = sisu[col].astype(str).str.replace(',', '.').astype(float)
    
    colunas_com_nan = sisu.columns[sisu.isnull().any()].tolist()

    if len(colunas_com_nan) > 0:
        if chave in ["2016-1", "2016-2"]:
            sisu['nu_notacorte'] = pd.to_numeric(sisu['nu_notacorte'], errors='coerce')
            sisu['nu_notacorte'] = sisu.groupby('nome_curso')['nu_notacorte'].transform(lambda x: x.fillna(x.mean()))

        elif chave in ["2018-1", "2021-2", "2022-1", "2022-2"]:
            sisu['NOTA_CORTE'] = pd.to_numeric(sisu['NOTA_CORTE'], errors='coerce')
            sisu['NOTA_CORTE'] = sisu.groupby('NOME_CURSO')['NOTA_CORTE'].transform(lambda x: x.fillna(x.mean()))

        else:
            sisu['NU_NOTACORTE'] = pd.to_numeric(sisu['NU_NOTACORTE'], errors='coerce')
            sisu['NU_NOTACORTE'] = sisu.groupby('NO_CURSO')['NU_NOTACORTE'].transform(lambda x: x.fillna(x.mean()))


    for col in colunas:
        if col in enem.columns:
            enem.loc[:, col] = enem[col].round(2).astype(float)
        if col in sisu.columns:
            sisu.loc[:, col] = sisu[col].round(2).astype(float)

    return sisu, enem

In [8]:
def mesclar_base(sisu, enem):
    base_mesclada = pd.merge(sisu, enem, on=['CH', 'CN', 'MT', 'LC', 'REDACAO'], how='inner')
    
    return base_mesclada

In [9]:
todas_bases = []

for ano, semestre in edicoes:
    chave = f"{ano}-{semestre}"

    endereco_ENEM = f'./DADOS/ENEM/microdados_enem_{ano-1}/DADOS/microdados_enem_{ano-1}.csv'
    
    endereco_SISU = f'./DADOS/SISU/ListagemChamadaRegular_{ano}-{semestre}/ListagemChamadaRegular_{ano}-{semestre}.csv'
    delimiter_SISU = bases_sisu[chave]["delimiter"]
    encoding_SISU = bases_sisu[chave]["encoding"]
    
    dados_enem = pd.read_csv(endereco_ENEM, delimiter=";", encoding="cp1252")
    dados_sisu = pd.read_csv(endereco_SISU, delimiter=delimiter_SISU, encoding=encoding_SISU, low_memory=False)
    

    if cod_instituicao != None:
        dados_sisu = filtrar_instituicao(chave, dados_sisu, cod_instituicao)

    if cod_campus != None:
        dados_sisu = filtrar_campus(chave, dados_sisu, cod_campus)

    sisu, enem = selecionar_colunas(chave, dados_sisu, dados_enem)

    sisu, enem = processar_base(chave, sisu, enem)

    base_mesclada = mesclar_base(sisu, enem)

    print(f'Base {ano}-{semestre} concluída!')
    print(f' - Nesta edição a instituição ou campus obteve {len(sisu)} inscrições e a base mesclada tem {len(base_mesclada)} linhas')

    colunas_padronizadas = ['CN', 'CH', 'LC', 'MT', 'REDACAO', 'NOTA_GERAL', 'NOTA_CORTE', 'SEXO_S', 'CURSO', 'DATA DE NASCIMENTO', 'UF', 'MUNICIPIO', 'APROVADO','ANO', 'EDICAO', 'SIGLA_IES', 'CAMPUS','SEXO_E', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q022', 'Q024', 'Q025']
    base_mesclada.columns = colunas_padronizadas

    todas_bases.append(base_mesclada) 

Base 2016-1 concluída!
 - Nesta edição a instituição ou campus obteve 70758 inscrições e a base mesclada tem 70658 linhas
Base 2016-2 concluída!
 - Nesta edição a instituição ou campus obteve 12389 inscrições e a base mesclada tem 12389 linhas
Base 2017-1 concluída!
 - Nesta edição a instituição ou campus obteve 67060 inscrições e a base mesclada tem 66877 linhas


In [10]:
Base_Final = pd.concat(todas_bases, ignore_index=True)
Base_Final.insert(0, 'ID', range(1, len(Base_Final) + 1))

In [11]:
Base_Final.to_excel('./Base.xlsx', index=False)